<a href="https://colab.research.google.com/github/Syedzamin07/cost-optimized-credit-card-fraud-detection/blob/main/02_data_understanding_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Understanding & Sanity Check

**Purpose:**
This notebook performs an initial inspection of the raw dataset to validate its structure, volume, and business constraints.

**Scope:**
- Load raw data.
- Verify dimensional shape (volume of transactions).
- Quantify class imbalance (fraud vs. legitimate).
- Document business context regarding feature anonymization.

**Note:** No modeling, scaling, feature engineering, or resampling is performed in this notebook. The goal is strictly to ground our downstream decisions in data reality.

In [8]:
import pandas as pd
import numpy as np

# Load the dataset
# Using the direct URL for reproducibility
url = "https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv"

try:
    df = pd.read_csv(url)
    print("✅ Dataset loaded successfully.")
except Exception as e:
    print(f"❌ Error loading data: {e}")

✅ Dataset loaded successfully.


In [9]:
# 1. Operational Volume Check
rows, cols = df.shape
print(f"Total Transactions: {rows:,}")
print(f"Total Features:     {cols}")

# Quick look at the data structure to confirm V1-V28 and Time/Amount exist
df.head()

Total Transactions: 284,807
Total Features:     31


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
# 2. Class Imbalance Check
# 0 = Legitimate, 1 = Fraud
counts = df['Class'].value_counts()
fraud_count = counts[1]
legit_count = counts[0]
fraud_ratio = (fraud_count / len(df)) * 100

print(f"Legitimate Transactions: {legit_count:,}")
print(f"Fraudulent Transactions: {fraud_count:,}")
print(f"Fraud Ratio:             {fraud_ratio:.4f}%")

Legitimate Transactions: 284,315
Fraudulent Transactions: 492
Fraud Ratio:             0.1727%


### 🔎 Business Context: Feature Anonymization (V1–V28)

The dataset contains features named `V1` through `V28`.

**Why are they anonymized?**
- **Privacy & Regulation:** Banks are strictly regulated (GDPR, CCPA, PCI-DSS) and cannot share Personally Identifiable Information (PII) like User IDs, Merchant Names, or Locations in public datasets.
- **PCA Transformation:** These features are the Principal Components obtained from a PCA transformation of the original data. This reduces dimensionality while preserving the variance (information) needed to distinguish patterns.

**Implications for this Project:**
1.  **Semantic Limits:** We cannot interpret features semantically (e.g., we won't know if `V4` represents "Distance from Home").
2.  **Behavioral Utility:** Despite the lack of names, these vectors still preserve **behavioral deviations**. Fraud detection relies on identifying mathematical outliers in this vector space, rather than simple rules like "blocked country."
3.  **No Feature Engineering:** Since the features are already transformed, traditional domain-specific feature engineering is not applicable here. We will rely on the raw `Time` and `Amount` features alongside these vectors.